<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler 

In [41]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference
def sigmoid(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = 1.0 if item > 0.0 else 0.0 
  return output

def relu(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = item if item > 0.0 else 0.0 
  return output

In [59]:
class Layer():
  def __init__(self, name, units, input_dim):
    self.name = name
    self.w = [[random.uniform(-2,2) for _ in range(input_dim)] for _ in range(units)]
    self.b = [0 for _ in range(units)]
    self.output = [bias for bias in self.b]
  def calculate_output(self, inputs, activation=sigmoid):
    self.output = [bias for bias in self.b]
    for row_index, col in enumerate(self.w):
      for col_index, weight in enumerate(col):
        self.output[row_index] += weight * inputs[col_index]
    self.output = activation(self.output)
    return self.output 
  def update_weights(self, error, inputs, learning_rate):
    error_out = 0
    for row_index, col in enumerate(self.w):
      for col_index, weight in enumerate(col):
        self.w[row_index][col_index] = self.w[row_index][col_index] + (learning_rate * error * inputs[col_index])
        self.b[row_index] = self.b[row_index] + (learning_rate * error)
    for input_idx, input in enumerate(inputs):
      error_out += sum([error * weight[input_idx] for weight_idx, weight in enumerate(self.w)])
    return error_out

In [55]:
def reset_layers():
  layer1 = Layer("Hidden1", 3, 2)
  layer2 = Layer("Output", 3, 3)
  layers = [layer1, layer2]
  return layers

In [5]:
iris = load_iris()

iris_columns = ['sepal_len', 'sepal_width', 'petal_length', 'petal_width', 'target']
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris_columns)

In [51]:
from sklearn.model_selection import train_test_split
# X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
X = df[['sepal_width', 'petal_length']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)


scaler_training = StandardScaler() 
scaler_training.fit(X_train)
scaled_values_training = scaler_training.transform(X_train)
scaled_values_testing = scaler_training.transform(X_test)

X_train = pd.DataFrame(scaled_values_training, columns=[*X_train.columns.values])
X_test = pd.DataFrame(scaled_values_testing, columns=[*X_test.columns.values])


X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [7]:
import random
random.seed(34)
selected_rows = random.sample(range(0,119),   5)

In [8]:
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2])
lb.transform([0])


array([[1, 0, 0]])

In [9]:
def categorical_cross_entropy(actual, predicted):
	sum_score = 0.0
	for i in range(len(actual)):
			sum_score += actual[i] * math.log(1e-15 + predicted[i])
	mean_sum_score = (1.0 / len(actual)) * sum_score
	return -mean_sum_score

In [60]:
def train_weights_using_mini_batch_gradient_descent(X_train, y_train, num_epochs, learning_rate, layers):
  # weights for all the features 
  index = 0;
  error = 0
  mini_batch = X_train.loc[selected_rows, :]
  y_train = y_train.loc[selected_rows, :]
  while index < num_epochs:
    print("Iteration %d"%(index+1))
    for rowIndex, row in mini_batch.iterrows():
      # print(y_train.loc[rowIndex,'target'])
      output = row
      for layer_idx, layer in enumerate(layers):
        if layer_idx == len(layers) - 1:
          output = layer.calculate_output(output, softmax)
        else:
          output = layer.calculate_output(output, relu)
      # print(output)
      # print(lb.transform([y_train.loc[rowIndex,'target']]))
      # print(categorical_cross_entropy(lb.transform([y_train.loc[rowIndex,'target']])[0], output))
      error = log_loss([y_train.loc[rowIndex,'target']], [output], labels=[0.0, 1.0, 2.0])
      print("Entropy based loss: %f"% (error))
      if error != 0:
        layer1_error = layers[1].update_weights(error, layers[0].output, learning_rate)
        layers[0].update_weights(layer1_error, row, learning_rate)
      print("New Weights of output layer", layers[1].w)
      print("New Weights of hidden layer", layers[0].w)
    # for layer_idx, layer in enumerate(layers):
    #   layer.update_weights()
    index = index + 1
  # df_plot = pd.DataFrame(df_plot, columns=['mse', 'epoch'])
  return

layers = reset_layers()
print("Creating model using training data")
train_weights_using_mini_batch_gradient_descent(X_train, y_train, 30, 0.01, layers)

Creating model using training data
Iteration 1
Entropy based loss: 1.069691
New Weights of output layer [[0.935065127354219, 0.4514718502417332, 1.3184982813309336], [-1.7722032810881396, 1.667270311591643, 0.3857743145198533], [0.5812496134286915, 1.1841164152291896, -1.6973198750034157]]
New Weights of hidden layer [[0.8125920418298592, 1.9288376140209293], [0.3383525543447529, 0.055146235745461035], [-0.33026606056999946, 1.6250216191531472]]
Entropy based loss: 4.833482
New Weights of output layer [[0.98262776925118, 0.4514718502417332, 1.3899782104505212], [-1.7246406391911786, 1.667270311591643, 0.4572542436394409], [0.6288122553256525, 1.1841164152291896, -1.6258399458838282]]
New Weights of hidden layer [[0.7086954789637538, 2.0511351739689463], [0.2344559914786476, 0.17744379569347796], [-0.4341626234361048, 1.7473191791011642]]
Entropy based loss: 1.344669
New Weights of output layer [[0.98262776925118, 0.45624607771663567, 1.3899782104505212], [-1.7246406391911786, 1.6720445

In [61]:
for rowIndex, row in X_test.iterrows():
  print(rowIndex)
  for layer_idx, layer in enumerate(layers):
      output = row
      
      if layer_idx == len(layers) - 1:
        output = layer.calculate_output(output, softmax)
      else:
        output = layer.calculate_output(output, relu)
  print(output)
  print((lb.transform([y_test.loc[rowIndex, 'target']])[0]))

0


IndexError: ignored